<div style="width: 38.5%;">
    <p><strong>City College of San Francisco</strong><p>
    <hr>
    <p>MATH 108 - Foundations of Data Science</p>
</div>

# Lecture 31: Least Squares

Associated Textbook Sections: [15.3, 15.4](https://inferentialthinking.com/chapters/15/3/Method_of_Least_Squares.html)

## Outline

* [Least Squares](#Least-Squares)
* [Numerical Optimization](#Numerical-Optimization)
* [Nonlinear Regression](#Nonlinear-Regression)

## Set Up the Notebook

In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about in this course.

def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope + intercept
    plots.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_table(r):
    """
    Generate a table of 1000 x,y data points in standard units
    whose correlation is approximately equal to r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)

def slope(t, x, y):
    return correlation(t, x, y) * np.std(t.column(y)) / np.std(t.column(x)) 

def intercept(t, x, y):
    return np.mean(t.column(y)) - slope(t, x, y) * np.mean(t.column(x)) 

---

## Least Squares

### Demonstration

In [ ]:
demographics = Table().read_table('./data/district_demographics2016.csv')
demographics.scatter('College%', 'Median Income')
plots.title('USA Congressional Districts, 2016')
plots.show()

In [ ]:
demographics

Based only on the above graph, which must be true?
1. Going to college causes people to get higher incomes.
2. For any district, having more college-educated people live there causes median incomes to rise.
3. For any district, having a higher median income causes more college-educated people to move there.

### Error in Estimation

* `error = actual value - estimate`
* Typically, some errors are positive and some negative
* To measure the rough size of the errors
    * square the errors to eliminate cancellation
    * take the mean of the squared errors
    * take the square root to fix the units
    * root mean square error (rmse)

### Demo: Regression line vs other lines

Use the following functions with the 2016 demographics data to explore the relationship between Median Income and College completion within the US voting districts.

In [ ]:
def demographics_errors(slope, intercept):
    # Use four convenient points from the original data
    sample = [[14.7, 33995], [19.1, 61454], [50.7, 71183], [59.5, 105918]]
    demographics.scatter('College%', 'Median Income', alpha=0.5)
    xlims = make_array(5, 75)
    # Plot a line with the slope and intercept you specified:
    plots.plot(xlims, slope * xlims + intercept, lw=4)
    # Plot red lines from each of the four points to the line
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=4)

In [ ]:
def fitted_values(t, x, y):
    """Return an array of the regressions estimates at all the x values"""
    a = ...
    b = ...
    return ...

In [ ]:
demographics = demographics.drop(
    'State', 'District')
demographics.show(5)

In [ ]:
demographics.scatter('College%', 'Median Income')

In [ ]:
correlation(demographics, 'College%', 'Median Income')

Determine the slope and intercept for the linear regression line to predict median income values from college% values.

In [ ]:
regression_slope = slope(demographics, 'College%', 'Median Income')
regression_intercept = intercept(demographics, 'College%', 'Median Income')
regression_slope, regression_intercept

In [ ]:
predicted = fitted_values(demographics, 'College%', 'Median Income')

In [ ]:
demographics = demographics.with_column(
    'Linear Prediction', predicted)
demographics.scatter('College%')

Analyze the errors formed when applying the predictions on the provided `College%` data values.

In [ ]:
actual = demographics.column('Median Income')
errors = ...

In [ ]:
demographics.with_column('Error', errors)

In [ ]:
np.mean(errors)

In [ ]:
np.mean(errors ** 2) ** 0.5

In [ ]:
demographics_errors(regression_slope, regression_intercept)

In [ ]:
# takes any slope, any intercept

demographics_errors(1500, 20000)

In [ ]:
demographics_errors(-1000, 75000)

### Least Squares Line

* Minimizes the root mean squared error (rmse) among all lines
* Equivalently, minimizes the mean squared error (mse) among all lines
* Common Names:
    * "Best fit" line
    * Least squares line
    * Regression line


### Demo: Root Mean Square Error

Explore the root mean square error through the demographics relationship.

In [ ]:
def show_demographics_rmse(slope, intercept):
    demographics_errors(slope, intercept)
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    prediction = slope * x + intercept
    mse = np.mean((y - prediction) ** 2)
    print("Root mean squared error:", round(mse ** 0.5, 2))

In [ ]:
show_demographics_rmse(-1000, 75000)

In [ ]:
show_demographics_rmse(1500, 20000)

In [ ]:
show_demographics_rmse(regression_slope, regression_intercept)

---

## Numerical Optimization

### Numerical Optimization

* Numerical minimization is approximate but effective
* Lots of machine learning uses numerical minimization
* If the function `mse(a, b)` returns the mse of estimation using the line `estimate = ax + b`, then `minimize(mse)` returns array `[slope, intercept]`
* the values are the slope and the intercept of the line that minimizes the mse among all possible lines.

### Demo: Numerical Optimization

Explore the concept of numerical optimization.

In [ ]:
x = np.arange(1, 3, 0.1)
y = (x-2)**2 + 3
Table().with_columns('x', x, 'y', y).plot('x')

In [ ]:
def f(x):
    return ...

In [ ]:
...

In [ ]:
x = np.arange(-1.5, 1.5, 0.05)
y2 = 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 
Table().with_columns('x', x, 'y', y2).plot('x')

In [ ]:
def complicated_function(x):
    return ...

In [ ]:
...

### Demo: Minimizing RMSE

Show how minimizing on the demographics rmse produces the same slope and intercept values found above. 

In [ ]:
def demographics_rmse(any_slope, any_intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = ...
    return ...

In [ ]:
demographics_rmse(1500, 20000)

In [ ]:
demographics_rmse(-1000, 75000)

In [ ]:
...

In [ ]:
make_array(regression_slope, regression_intercept)

## Nonlinear Regression

### Nonlinear Regression

In [ ]:
shotput = Table.read_table('data/shotput.csv')
shotput

In [ ]:
shotput.scatter('Weight Lifted')

In [ ]:
def shotput_linear_rmse(any_slope, any_intercept):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = any_slope*x + any_intercept
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
best_line = ...
best_line

In [ ]:
weights = shotput.column(0)

In [ ]:
linear_fit = ...

shotput.with_column(
    'Best Line', linear_fit
).scatter(0)

#### Quadratic Function

$$
f(x) ~=~ ax^2 + bx + c
$$
for constants $a$, $b$, and $c$.

In [ ]:
def shotput_quadratic_rmse(a, b, c):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = a*(x**2) + b*x + c
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
best_quad = ...
best_quad

In [ ]:
# x = weight lifted = 100 kg
# Then predicted shot put distance:

...

In [ ]:
quad_fit = ...

In [ ]:
shotput.with_column('Best Quadratic Curve', quad_fit).scatter(0)

---

<footer>
    <p>Adopted from UC Berkeley DATA 8 course materials.</p>
    <p>This content is offered under a <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/">CC Attribution Non-Commercial Share Alike</a> license.</p>
</footer>